Mount the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Install Neccessary libraries

In [2]:
!pip -q install langchain openai tiktoken faiss-gpu pypdf rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.25.2
    Uninstalling openai-1.25.2:
      Successfully uninstalled openai-1.25.2


Import the necessary libraries

In [4]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [5]:
# Load and process the PDF document
pdf_path = '/content/drive/MyDrive/pdfs/1Q23-EPR-with-Tables-FINAL.pdf'
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [6]:
import os

os.environ["OPENAI_API_KEY"] = " "

In [8]:
# Split the text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [9]:
# Create vector store
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY )
vectorstore = FAISS.from_documents(texts, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
# Create conversation chain
llm = OpenAI(temperature=0.1, model_name="gpt-4")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [11]:
# Interactive conversation loop
from rouge import Rouge
rouge = Rouge()

while True:
    query = input("User: ")
    if query.lower() == 'exit':
        break

    result = conversation_chain({"question": query})
    generated_answer = result["answer"]

    reference_answer = input("Please provide the reference answer: ")

    # Calculate ROUGE scores
    scores = rouge.get_scores(generated_answer, reference_answer)
    rouge_1 = scores[0]['rouge-1']['f']
    rouge_2 = scores[0]['rouge-2']['f']  # Include ROUGE-2 score
    rouge_l = scores[0]['rouge-l']['f']  # Include ROUGE-L score

    print(f"Assistant: {generated_answer}")
    print(f"Reference: {reference_answer}")
    print(f"ROUGE-1 Score: {rouge_1}")
    print(f"ROUGE-2 Score: {rouge_2}")  # Print ROUGE-2 score
    print(f"ROUGE-L Score: {rouge_l}")  # Print ROUGE-L score
    print()


User:  A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Please provide the reference answer: To meet the Extended Producer Responsibility (EPR) scheme requirement of an 80% recovery target, the company should aim to recover: Required Recovery = 500 tons * 80% = 400 tons. Thus, the company should aim to recover 400 tons of plastic packaging each year to meet the EPR requirement.
Assistant: The company should aim to recover 400 tons of plastic packaging each year to meet the EPR requirement.
Reference: To meet the Extended Producer Responsibility (EPR) scheme requirement of an 80% recovery target, the company should aim to recover: Required Recovery = 500 tons * 80% = 400 tons. Thus, the company should aim to recover 400 tons of plastic packaging each year to meet the EPR requirement.
ROUGE-1 Score: 0.6399999955120002
ROUGE-2 Score: 0.5423728772536627
ROUGE-L Score: 0.6399999955120002

User: What is the report quarter and when did it end?
Please provide the reference answer: The reported quarter was the first quarter of fiscal 2023. It ended 